In [ ]:
import os
import cv2
import numpy as np
import json
from tqdm import tqdm
from skimage.draw import polygon2mask
import shutil

In [ ]:
json_dir = './val/label/damage'
img_dir = './val/img/damage'
file_names = os.listdir(json_dir)
output_ann_dir = './val/ann_dir/train/'
output_img_dir = './val/img_dir/train/'
os.makedirs(output_ann_dir, exist_ok=True)
os.makedirs(output_img_dir, exist_ok=True)

In [ ]:
# Whole type
damage_type = {'Scratched':1, 'Crushed':2, 'Breakage':3, 'Separated':4} # num : priority

for file_name in tqdm(file_names):
    with open(os.path.join(json_dir, file_name)) as f:
        data = json.load(f)

    height, width = data['images']['height'], data['images']['width']
    result_array = np.zeros((height, width))
    n = len(data['annotations'])
    
    for i in range(n):
        polygons = data['annotations'][i]['segmentation']
        type = damage_type[data['annotations'][i]['damage']]
        for j in range(len(polygons)):
            for k in range(len(polygons[j])):
                tmp = np.array(polygons[j][k]).squeeze()
                poly = [[p[1], p[0]] for p in tmp]
                mask = polygon2mask((height,width), poly) * type
                if k == 0:
                    result_array = np.maximum(result_array, mask)
                else:
                    result_array[mask == 1] = 0
                
                
    cv2.imwrite(os.path.join(output_ann_dir, file_name.split('.')[0]+'.png'), result_array.astype(np.uint8))
                    


In [ ]:
# Select type
damage_type = 'Separated'

for file_name in tqdm(file_names):
    with open(os.path.join(json_dir, file_name)) as f:
        data = json.load(f)

    height, width = data['images']['height'], data['images']['width']
    result_array = np.zeros((height, width))
    n = len(data['annotations'])
    save_flag = False
    for i in range(n):
        polygons = data['annotations'][i]['segmentation']
        type = data['annotations'][i]['damage']
        if type != damage_type:
            continue
        if not save_flag:
            save_flag = True
        for j in range(len(polygons)):
            for k in range(len(polygons[j])):
                tmp = np.array(polygons[j][k]).squeeze()
                poly = [[p[1], p[0]] for p in tmp]
                mask = polygon2mask((height,width), poly)
                if k == 0:
                    result_array = np.maximum(result_array, mask)
                else:
                    result_array[mask == 1] = 0

    if save_flag:
        name = file_name.split('.')[0]
        cv2.imwrite(os.path.join(output_ann_dir, name+'.png'), result_array.astype(np.uint8))
        shutil.copy(os.path.join(img_dir, name+'.jpg'), output_img_dir)
                    
